# Fst Catalog

In [ ]:
import datetime
from glob import glob
from pathlib import Path

import numpy as np
from osgeo import gdal
import xarray as xr

import fstcatalog
import rioxarray

## Get some files to process

In [ ]:
current_date = datetime.datetime.now()
year_month_day_string = current_date.strftime('%Y%m%d')

filter = '00_01*'
base_path1 = Path('/home/smco500/cmcprod/ppp5/suites/gdps/g1/gridpt.usr/prog/eta')
base_path2 = Path('/home/smco500/cmcprod/ppp5/suites/gdps/g1/gridpt.usr/prog/diag')
base_path3 = Path('/home/smco500/cmcprod/ppp5/suites/gdps/g1/gridpt.usr/prog/pres')
base_path4 = Path('/home/smco500/cmcprod/ppp5/suites/rdps/r1/gridpt.usr/prog/eta')
base_path5 = Path('/home/smco500/cmcprod/ppp5/suites/rdps/r1/gridpt.usr/prog/diag')
base_path6 = Path('/home/smco500/cmcprod/ppp5/suites/rdps/r1/gridpt.usr/prog/pres')
base_path7 = Path('/home/smco500/cmcprod/ppp5/suites/hrdps_national/n1/wxelements')
base_path8 = Path('/home/smco500/cmcprod/ppp5/suites/hrdps_national/n1/sampling')
base_path9 = Path('/space/hall5/sitestore/eccc/crd/ccmr/users/min000/fstd2nc-testfiles/')
# files = glob(f'{base_path9}/*')
files = glob(f'{base_path1}/{year_month_day_string}{filter}')
files.extend(glob(f'{base_path2}/{year_month_day_string}{filter}'))
files.extend(glob(f'{base_path3}/{year_month_day_string}{filter}'))
files.extend(glob(f'{base_path4}/{year_month_day_string}{filter}'))
files.extend(glob(f'{base_path5}/{year_month_day_string}{filter}'))
files.extend(glob(f'{base_path6}/{year_month_day_string}{filter}'))
files.extend(glob(f'{base_path7}/{year_month_day_string}{filter}'))
files.extend(glob(f'{base_path8}/{year_month_day_string}{filter}'))


## Get Catalog of valid fst files

In [ ]:
cat = fstcatalog.FstCatalog(files).catalog()

## Inspect data

In [ ]:
cat.df

# Get a specific xarray Dataset by record nomvar

In [ ]:
filter = (cat.df.nomvar=='TT') & (cat.df.path.str.contains('pres')) & (cat.df.crs_cf_name == 'rotated_pole') & (cat.df.level_unit == 'mb')
display(cat.df.loc[filter])

xr.merge([cat.get_dataset(i) for i in cat.df.loc[filter].index.tolist()])

## Get a specific xarray DataSet by id

In [ ]:
cat.get_dataset(441)


## Get a specific xarray DataSet with its projection information by id

In [ ]:
ds, crs_name, proj4= cat.get_dataset_with_crs_info(441)
display(ds)
display(crs_name)
display(proj4)

In [ ]:
# display(ds.isel(time=0, pres=0))
ds_subset = ds.isel(time=0, pres=0)
src_crs = ds_subset['WKT'].data.item()
# # Reproject to WGS84
ds_subset = ds_subset.drop(['time','pres','leadtime','reftime', 'proj4','rotated_pole','lat','lon'])
ds_subset.rio.write_crs(ds_subset['WKT'].data.item(), inplace=True)
# display(ds_subset)
ds_subset = ds_subset.rio.reproject("EPSG:4326")
ds_subset.TT

## Plot a field interactivly with hvplot

In [ ]:
cat.get_hvplot(441)

## Display class attributes

In [ ]:
cat.df

In [ ]:
cat.files[0:10]

## View the decoded attributes

In [ ]:
cat.advanced_view()

## View like the voir command

In [ ]:
cat.voir_view()

In [ ]:
import xarray as xr
import numpy as np
from osgeo import gdal

In [ ]:
display(ds.proj4.data.item())

In [ ]:
slice_data = ds['TT']
# slice_data.coords

def drop_dims(da):
    # Check if 'rlat' and 'rlon' are in the coordinates
    if 'rlat' in da.coords and 'rlon' in da.coords:
        # Check if 'lat' and 'lon' are in the dimensions
        if 'lat' in da.coords and 'lon' in da.coords:
            # Remove 'lat' and 'lon' from the dimensions
            da = da.drop(['lat', 'lon'])
    return da

slice_data = drop_dims(slice_data)
# slice_data
unique_time = np.unique(slice_data['time'].values)
unique_pres = np.unique(slice_data['pres'].values)
for t in unique_time:
    for p in unique_pres:
        # Extract the 2D array corresponding to this time and pressure level
        # display(t,p,slice_data.sel(time=t, pres=p).drop(['time','pres']))
        arr = slice_data.sel(time=t, pres=p).values
        
        # Define the output file name
        filename = f'/home/sbf000/ss6/tiffs/TT_time{t}_pres{p}.tif'
        
        # Create a GeoTIFF file with the array data
        driver = gdal.GetDriverByName('GTiff')
        out_ds = driver.Create(filename, arr.shape[1], arr.shape[0], 1, gdal.GDT_Float32)
        out_ds.SetGeoTransform((ds.rlon[0], ds.rlon[1]-ds.rlon[0], 0, ds.rlat[-1], 0, ds.rlat[1]-ds.rlat[0]))
        out_ds.SetProjection(ds.proj4.data.item()) #'+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs'
        out_band = out_ds.GetRasterBand(1)
        out_band.WriteArray(arr)
        out_band.FlushCache()
    


            

In [ ]:
import asyncio
import os 
current_date = datetime.datetime.now()
year_month_day_string = current_date.strftime('%Y%m%d')

filter = '00_*'
base_path = Path('/home/smco500/cmcprod/ppp5/suites/') #gdps/g1/gridpt.usr/prog/
dirs = [d for d in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, d))]
dirs
# files = glob(f'{base_path}/**/{year_month_day_string}{filter}', recursive=True)
# # files = glob(f'{base_path}/**/[0-9][0-9][0-9][0-9][0-9][0-9][0-9][0-9][0-9][0-9]_[0-9][0-9][0-9]', recursive=True)
# files = np.unique([str(Path(f).parent).replace('/home/smco500/cmcprod/ppp5/suites/', '') for f in files ])
# files


In [60]:
import asyncio
import multiprocessing
import os
from glob import glob
from pathlib import Path
import numpy as np
from datetime import datetime, timedelta
import pandas as pd
import fstpy



async def search_files(directory, pattern):
    files = await asyncio.get_event_loop().run_in_executor(None, lambda: glob(f"{directory}/**/{pattern}", recursive=True))
    return files


def filter_fst_files(files, num_proc: int = 8):
    with multiprocessing.Pool(min(num_proc,len(files))) as pool:
        filtered_files = pool.map(fstpy.std_io.maybeFST, files)
        files = np.where(filtered_files, files,'').tolist()
        files = [f for f in files if f != '']
    return files
        


def get_current_utc_datetime():
    current_date = datetime.utcnow()
    year_month_day_string = current_date.strftime('%Y-%m-%d')
    current_datetime = pd.to_datetime(year_month_day_string, format='%Y-%m-%d')
    current_datetime = current_datetime.replace(hour=0, minute=0, second=0, microsecond=0)
    return current_datetime


def create_dataframe(current_datetime, results):
    df = pd.DataFrame(results)
    df['suffix'] = pd.Series([f[15:]  if len(f) > 14 else '' for f in df.file])
    year = df.file.str[0:4]
    month = df.file.str[4:6]
    day = df.file.str[6:8]
    hour = df.file.str[8:10]
    df['pass'] = df.file.str[11:14]
    date_str = year + '-' + month + '-' + day + ' ' + hour
    df['dateo'] = pd.to_datetime(date_str)
    df['datev'] = df['dateo'] + pd.to_timedelta(df['pass'].astype(int),'h')
    df['delta'] = (df.iloc[0].dateo - current_datetime).astype('timedelta64[h]')

    return df



ops_dir = Path('/home/smco500/cmcprod/ppp5/suites')
base_path = ops_dir / Path('gdps/g1/gridpt.usr/prog')
directories = [base_path / d for d in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, d))]
display(directories)
pattern = '[0-9][0-9][0-9][0-9][0-9][0-9][0-9][0-9][0-9][0-9]_[0-9][0-9][0-9]*'
tasks = [asyncio.create_task(search_files(directory, pattern)) for directory in directories]
files = (await asyncio.gather(*tasks))
files = [item for sublist in files for item in sublist]
# files = filter_fst_files(files)


current_datetime = get_current_utc_datetime()
display(current_datetime)

# fnames = [Path(f).stem for f in files]
results = [{'base': base_path, 'model': str(Path(f).parent).replace(f'{ops_dir}/','').split('/')[0], 'sub_path': str(Path(f).parent).replace(f'{base_path}/',''), 'file':str(Path(f).stem).replace(f'{base_path}/','')} for f in files]

df = create_dataframe(current_datetime, results)
# display(df)

# df.sort_values(['sub_path', 'datev', 'delta'],inplace=True)

# nfile = [(current_datetime + timedelta(hours = int(d))).strftime('%Y%m%d%H') for d in df.delta]

# df['nfile'] = pd.Series(nfile)
display(df)
current_datetime, df.iloc[0].dateo, df.iloc[0].dateo - (current_datetime), (current_datetime) + (df.iloc[0].dateo - (current_datetime))


[PosixPath('/home/smco500/cmcprod/ppp5/suites/gdps/g1/gridpt.usr/prog/eta'),
 PosixPath('/home/smco500/cmcprod/ppp5/suites/gdps/g1/gridpt.usr/prog/oce'),
 PosixPath('/home/smco500/cmcprod/ppp5/suites/gdps/g1/gridpt.usr/prog/pres'),
 PosixPath('/home/smco500/cmcprod/ppp5/suites/gdps/g1/gridpt.usr/prog/diag'),
 PosixPath('/home/smco500/cmcprod/ppp5/suites/gdps/g1/gridpt.usr/prog/sampling')]

Timestamp('2023-03-24 00:00:00')

AttributeError: 'Timedelta' object has no attribute 'astype'